## QLearning利用tkinter实现

利用探索者（红色）从左上角往四个方向探索宝藏<br>
* ACTIONS：向左、向右、向上和向下<br>
* N_STATES：不定个数的状态<br>
* 黑色代表深渊（reward=-1），黄色代表宝藏（reward=+1）

![](picture1.png)

### 强化学习的大脑
同第一节的那些功能，但是多了一个check_state_exist，由于网格的状态太多了，因此没有出现过的状态不予记录（建立空的Q表格），而是在出现后再加入Q表格中

In [9]:
import numpy as np
import pandas as pd


class QLearningTable:
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = actions  # a list
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)

    def choose_action(self, observation):
        self.check_state_exist(observation)
        # action selection
        if np.random.uniform() < self.epsilon:
            # choose best action
            state_action = self.q_table.loc[observation, :]
            # some actions may have the same value, randomly choose on in these actions
            action = np.random.choice(state_action[state_action == np.max(state_action)].index)
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action

    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)  # update

    def check_state_exist(self, state):
        if state not in self.q_table.index:
            # append new state to q table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0]*len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            )
            
action_space = ['u', 'd', 'l', 'r']
RL = QLearningTable(actions = action_space)
print(RL.q_table)

Empty DataFrame
Columns: [u, d, l, r]
Index: []


### 利用tkinter建立环境
* _build_maze：利用画布Cavas建立初始环境
* reset：刷新页面，并返回当前探索者的位置坐标（长度为4的list）
* step：根据行动返回下一状态、奖赏和是否继续游戏

In [18]:
import time
import sys
if sys.version_info.major == 2:
    import Tkinter as tk
else:
    import tkinter as tk


UNIT = 40   # pixels
MAZE_H = 4  # grid height
MAZE_W = 4  # grid width


class Maze(tk.Tk, object):
    def __init__(self):
        super(Maze, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.title('maze')
        self.geometry('{0}x{1}'.format(MAZE_H * UNIT, MAZE_H * UNIT))
        self._build_maze()

    def _build_maze(self): 
        self.canvas = tk.Canvas(self, bg='white',
                           height=MAZE_H * UNIT,
                           width=MAZE_W * UNIT)

        # create grids
        for c in range(0, MAZE_W * UNIT, UNIT):
            x0, y0, x1, y1 = c, 0, c, MAZE_H * UNIT
            self.canvas.create_line(x0, y0, x1, y1)
        for r in range(0, MAZE_H * UNIT, UNIT):
            x0, y0, x1, y1 = 0, r, MAZE_W * UNIT, r
            self.canvas.create_line(x0, y0, x1, y1)

        # create origin
        origin = np.array([20, 20])

        # hell
        hell1_center = origin + np.array([UNIT * 2, UNIT])
        self.hell1 = self.canvas.create_rectangle(
            hell1_center[0] - 15, hell1_center[1] - 15,
            hell1_center[0] + 15, hell1_center[1] + 15,
            fill='black')
        # hell
        hell2_center = origin + np.array([UNIT, UNIT * 2])
        self.hell2 = self.canvas.create_rectangle(
            hell2_center[0] - 15, hell2_center[1] - 15,
            hell2_center[0] + 15, hell2_center[1] + 15,
            fill='black')

        # create oval
        oval_center = origin + UNIT * 2
        self.oval = self.canvas.create_oval(
            oval_center[0] - 15, oval_center[1] - 15,
            oval_center[0] + 15, oval_center[1] + 15,
            fill='yellow')

        # create red rect
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')

        # pack all
        self.canvas.pack()

    def reset(self):
        self.update() # tkinter.update() 刷新页面
        time.sleep(0.5)
        self.canvas.delete(self.rect)
        origin = np.array([20, 20])
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')
        # return observation
        return self.canvas.coords(self.rect)

    def step(self, action):
        s = self.canvas.coords(self.rect)
        base_action = np.array([0, 0])
        if action == 0:   # up
            if s[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:   # down
            if s[1] < (MAZE_H - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:   # right
            if s[0] < (MAZE_W - 1) * UNIT:
                base_action[0] += UNIT
        elif action == 3:   # left
            if s[0] > UNIT:
                base_action[0] -= UNIT

        self.canvas.move(self.rect, base_action[0], base_action[1])  # move agent

        s_ = self.canvas.coords(self.rect)  # next state

        # reward function
        if s_ == self.canvas.coords(self.oval):
            reward = 1
            done = True
            s_ = 'terminal'
        elif s_ in [self.canvas.coords(self.hell1), self.canvas.coords(self.hell2)]:
            reward = -1
            done = True
            s_ = 'terminal'
        else:
            reward = 0
            done = False

        return s_, reward, done

    def render(self):
        time.sleep(0.1)
        self.update()


env = Maze()
env._build_maze()
env.mainloop()

先利用一个简单的操作，只进行操作1（向下），直至不能向下为止

In [ ]:
def update(): 
    for t in range(10):
        s = env.reset()
        while True:
            env.render()
            a = 1
            s, r, done = env.step(a)
            if done:
                break
env = Maze()
env.after(100, update) # tkinter.after() 注册一个n微秒后执行的函数update
env.mainloop()

### 主函数
* 更新环境
* 选择行动并得到状态
* 选择行动并得到下一步的状态和奖赏
* 学习并更新Q表格
* 更新状态

In [35]:
def update():
    for episode in range(20):
        # initial observation
        observation = env.reset()

        while True:
            # fresh env
            env.render()

            # RL choose action based on observation
            action = RL.choose_action(str(observation))

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            # RL learn from this transition
            RL.learn(str(observation), action, reward, str(observation_))

            # swap observation
            observation = observation_
            
            

            # break while loop when end of this episode
            if done:
                break
        print('\r\nQ-table:\n')
        print(RL.q_table)
        print('\r')
    # end of game
    print('game over')
    env.destroy()

![](picture2.gif)

In [36]:
env = Maze()
RL = QLearningTable(actions=list(range(env.n_actions)))
env.after(100, update)
env.mainloop()


Q-table:

                            0    1     2    3
[5.0, 5.0, 35.0, 35.0]    0.0  0.0  0.00  0.0
[45.0, 5.0, 75.0, 35.0]   0.0  0.0  0.00  0.0
[45.0, 45.0, 75.0, 75.0]  0.0  0.0 -0.01  0.0
terminal                  0.0  0.0  0.00  0.0


Q-table:

                            0     1     2    3
[5.0, 5.0, 35.0, 35.0]    0.0  0.00  0.00  0.0
[45.0, 5.0, 75.0, 35.0]   0.0  0.00  0.00  0.0
[45.0, 45.0, 75.0, 75.0]  0.0 -0.01 -0.01  0.0
terminal                  0.0  0.00  0.00  0.0
[5.0, 45.0, 35.0, 75.0]   0.0  0.00  0.00  0.0
[85.0, 5.0, 115.0, 35.0]  0.0  0.00  0.00  0.0


Q-table:

                             0     1     2    3
[5.0, 5.0, 35.0, 35.0]     0.0  0.00  0.00  0.0
[45.0, 5.0, 75.0, 35.0]    0.0  0.00  0.00  0.0
[45.0, 45.0, 75.0, 75.0]   0.0 -0.01 -0.01  0.0
terminal                   0.0  0.00  0.00  0.0
[5.0, 45.0, 35.0, 75.0]    0.0  0.00  0.00  0.0
[85.0, 5.0, 115.0, 35.0]   0.0 -0.01  0.00  0.0
[125.0, 5.0, 155.0, 35.0]  0.0  0.00  0.00  0.0


Q-table:

          